In [2]:
# import sys
# !{sys.executable} -m pip install Jinja2
import glacierml as gl
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
parameterization = '4'
method = '1'

2023-09-15 18:55:38.885893: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-15 18:55:38.929153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 18:55:39.528340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [27]:
def stats_builder(st1 = 1e-5, st2 = 1e4):
    
#     boundary = [(str(st1), str(st2))]
    
    col_list = [ 'Area', 
#                 'Slope','Lmax','Zmin',
#                 'Zmax',
                '0','Thick Diff',
                'Vol Diff'
#                'Farinotti Mean Thickness'
               ]
    name = ['mean',
            'median', 'min', 'max','IQR',
            'STD','count'
           ]

    df = pd.DataFrame( columns = col_list, index = name)
#     data = gl.load_notebook_data('4','first')
    data = pd.read_pickle('another_quick_pick.pkl')
    x = np.array(data['FMT'])
    y = np.array(data['0'])
    area = np.array(data['Area'])
    
    vol_f = (x / 1e3) * area
    vol = (y / 1e3) * area
    
    thick_f = vol_f / area * 1e3
    thick = vol / area * 1e3
    
    vol_diff = vol - vol_f
    data['Thick Diff'] = thick - thick_f
    data['Vol Diff'] = vol_diff
    data = data.iloc[np.where(
        (vol > st1) & (vol < st2)
    )]
    vol_sub = vol[np.where(
        (vol > st1) & (vol < st2)
    )]
    feat_list = col_list
    for i in feat_list:
        dft = data[i]
        upp = np.nanquantile(dft, 0.75)
        low = np.nanquantile(dft, 0.25)
#         print(np.quantile(dft, 0.75))
#         print(np.quantile(dft, 0.25))
        functions = [
            np.round(np.nanmean(dft), 3),
            np.round(np.nanmedian(dft), 3), 
            np.round(np.nanmin(dft), 3),
            np.round(np.nanmax(dft), 3),
            np.round(upp - low, 3),
            np.round(np.nanstd(dft),3),
            len(dft)
        ]
        for n, fn in zip(name, functions):
            df[i].loc[n] = fn
    df = df.rename(columns = {
        'Area':'Area (km$^2$)',
        'Slope':'Slope (deg)',
        'Lmax':'Max Length (m)',
        'Zmin':'Min Elevation (m)',
        '0':'Est Thick (m)',
        'Vol Diff':'Vol Diff (km$^3$)'
#         'Farinotti Mean Thickness':'Farinotti Thickness'
    })
    df = df.round(decimals = 3)
    return df, data, vol, vol_sub

In [54]:
stats, data, vol, vol_sub = stats_builder(1e-5, 1e4)
stats

,Area (km$^2$),Est Thick (m),Thick Diff,Vol Diff (km$^3$)
mean,3.273,48.812,20.568,-0.018
median,0.251,50.352001,21.845,0.004
min,0.01,5.049,-428.348,-468.126
max,7537.579,1102.133057,553.133,1673.292
IQR,0.681,32.219,31.444,0.011
STD,48.302,24.707001,22.976,7.153
count,215519,215519,215519,215519


In [56]:
tvol = sum((data['0'] / 1e3 * data['Area'])/1e3)
print(f'volume of total edf is {tvol}')
# print(stats.to_latex(float_format="%.2f"))

volume of total edf is 145.4971826374231


In [59]:
stats, data, vol, vol_sub = stats_builder(1e-4, 1e0)
stats

,Area (km$^2$),Est Thick (m),Thick Diff,Vol Diff (km$^3$)
mean,0.79,46.841999,21.733,0.007
median,0.238,49.597,22.576,0.004
min,0.01,5.135,-428.348,-3.858
max,31.518,394.953003,360.953,0.976
IQR,0.602,31.958,31.003,0.011
STD,1.584,19.226999,20.504,0.051
count,209715,209715,209715,209715


In [60]:
svol = np.round(sum((data['0'] / 1e3 * data['Area'])/1e3),3)
perc = np.round(svol / tvol * 100,2)
print(f'volume of subset edf is {svol}, {perc}% of global volume')
# print(stats.to_latex(float_format="%.2f"))

volume of subset edf is 9.568, 6.58% of global volume


In [61]:
stats, data, vol, vol_sub = stats_builder(1e0, 1e2)
stats

,Area (km$^2$),Est Thick (m),Thick Diff,Vol Diff (km$^3$)
mean,54.375,112.093002,-23.068,-2.154
median,27.271,101.859001,-13.117,-0.326
min,7.47,17.739,-302.797,-277.826
max,917.532,412.604004,269.604,38.68
IQR,39.942,39.115,45.662,1.589
STD,71.479,40.217999,45.869,9.583
count,5512,5512,5512,5512


In [62]:
svol = np.round(sum((data['0'] / 1e3 * data['Area'])/1e3),3)
perc = np.round(svol / tvol * 100,2)
print(f'volume of subset edf is {svol}, {perc}% of global volume')
# print(stats.to_latex(float_format="%.2f"))

volume of subset edf is 43.653, 30.0% of global volume


In [63]:
stats, data, vol, vol_sub = stats_builder(1e2, 1e4)
stats

,Area (km$^2$),Est Thick (m),Thick Diff,Vol Diff (km$^3$)
mean,1029.968,334.837006,9.378,27.887
median,688.885,318.294006,13.509,9.123
min,331.578,93.167,-252.833,-468.126
max,7537.579,1102.133057,553.133,1673.292
IQR,596.701,125.134,144.313,99.195
STD,955.749,114.871002,116.738,210.384
count,233,233,233,233


In [64]:
svol = np.round(sum((data['0'] / 1e3 * data['Area'])/1e3),3)
perc = np.round(svol / tvol * 100,2)
print(f'volume of subset edf is {svol}, {perc}% of global volume')
# print(stats.to_latex(float_format="%.2f"))

volume of subset edf is 92.276, 63.42% of global volume


In [ ]:
data

In [ ]:
data = data.dropna(subset = 'Farinotti Mean Thickness')

In [ ]:
import matplotlib.pyplot as plt




In [ ]:
np.sqrt(55000)

In [ ]:
plt.hist(data['Vol Diff'],75)
plt.yscale('log')

In [ ]:
f = data['Vol Diff']
plt.hist((f), 25)
plt.yscale('log')
plt.plot(
    [np.percentile(f,50),np.percentile(f,50)],
    [0,1e4], 'k--'
)
plt.plot(
    [np.percentile(f,25),np.percentile(f,25)],
    [0,1e4], 'k--'
)
plt.plot(
    [np.percentile(f,75),np.percentile(f,75)],
    [0,1e4], 'k--'
)
# plt.text(
    
    
#     (f).mean() - 0.5,
#     955, s = 'Median'
# )
# plt.text(
#     np.percentile(f,75) + .18 ,
#     900, s = 'Q3'
# )
# plt.text(
#     np.percentile(f, 25) - 0.76,
#     900, s = 'Q1'
# )

In [ ]:
data

In [ ]:
data['Thick Diff'] = data['Weighted Mean Thickness 1'] - data['Farinotti Mean Thickness']
data

In [ ]:
f = data['Thick Diff']
plt.hist(data['Thick Diff'], 25)
plt.yscale('log')
plt.plot(
    [np.percentile(f,50),np.percentile(f,50)],
    [0,1e5], 'k--'
)
plt.plot(
    [np.percentile(f,25),np.percentile(f,25)],
    [0,1e5], 'k--'
)
plt.plot(
    [np.percentile(f,75),np.percentile(f,75)],
    [0,1e5], 'k--'
)

In [ ]:
np.percentile(f,50)

In [ ]:
x = data['Farinotti Mean Thickness'] / 1e3 * data['Area']
y = data['Weighted Mean Thickness 1'] / 1e3 * data['Area']

np.nanmean(x - y)


In [ ]:
len(data)

In [ ]:
(len(data) / 216501) * 100

In [ ]:
(sum(vol_sub) / sum(vol)) * 100

In [ ]:
sum(vol) / 1e3

In [ ]:
sum(vol_sub) / 1e3